In [1]:
%matplotlib inline
%run ../../import_envs.py
print('probtorch:', probtorch.__version__, 
      'torch:', torch.__version__, 
      'cuda:', torch.cuda.is_available())

probtorch: 0.0+5a2c637 torch: 0.4.1 cuda: True


In [2]:
## Load dataset
data_path = "../gmm_dataset_c20k"
Data = torch.from_numpy(np.load(data_path + '/obs.npy')).float()

NUM_DATASETS, N, D = Data.shape
K = 3 ## number of clusters
SAMPLE_SIZE = 10
NUM_HIDDEN_LOCAL = 32

MCMC_SIZE = 10
BATCH_SIZE = 20
NUM_EPOCHS = 250
LEARNING_RATE = 5 * 1e-4
CUDA = torch.cuda.is_available()
PATH = 'soft-os-wsym'
DEVICE = torch.device('cuda:1')

Train_Params = (NUM_EPOCHS, NUM_DATASETS, SAMPLE_SIZE, BATCH_SIZE, CUDA, DEVICE, PATH)
Model_Params = (N, K, D, MCMC_SIZE)

In [3]:
from local_enc import *
from global_oneshot import *
from global_enc_v1 import *
## if reparameterize continuous variables
Reparameterized = False
# initialization
enc_z = Enc_z(K, D, NUM_HIDDEN_LOCAL, CUDA, DEVICE)
enc_eta = Enc_eta(K, D, CUDA, DEVICE, Reparameterized)
oneshot_eta = Oneshot_eta(K, D, CUDA, DEVICE, Reparameterized)

if CUDA:
    enc_z.cuda().to(DEVICE)
    enc_eta.cuda().to(DEVICE)
    oneshot_eta.cuda().to(DEVICE)
optimizer =  torch.optim.Adam(list(oneshot_eta.parameters())+list(enc_eta.parameters())+list(enc_z.parameters()),lr=LEARNING_RATE, betas=(0.9, 0.99))
models = (oneshot_eta, enc_eta, enc_z)

In [ ]:
from ag_ep import *
train(models, EUBO_init_eta, optimizer, Data, Model_Params, Train_Params)

epoch: 0\250 (140s),  symKL_DB_eta: 117.862,  symKL_DB_z: 135.843,  loss: -844.786,  ess: 3.887,  kl_eta_ex: 300.483,  kl_eta_in: 85.927,  kl_z_ex: 83.052,  kl_z_in: 18.699
epoch: 1\250 (145s),  symKL_DB_eta: 18.282,  symKL_DB_z: 26.771,  loss: -505.964,  ess: 5.485,  kl_eta_ex: 28.929,  kl_eta_in: 35.341,  kl_z_ex: 13.274,  kl_z_in: 3.802
epoch: 2\250 (151s),  symKL_DB_eta: 10.588,  symKL_DB_z: 12.184,  loss: -465.782,  ess: 6.200,  kl_eta_ex: 12.607,  kl_eta_in: 23.974,  kl_z_ex: 5.076,  kl_z_in: 1.738
epoch: 3\250 (151s),  symKL_DB_eta: 7.235,  symKL_DB_z: 7.255,  loss: -451.896,  ess: 6.791,  kl_eta_ex: 7.077,  kl_eta_in: 15.192,  kl_z_ex: 2.708,  kl_z_in: 1.036
epoch: 4\250 (148s),  symKL_DB_eta: 5.109,  symKL_DB_z: 4.036,  loss: -443.582,  ess: 7.334,  kl_eta_ex: 3.032,  kl_eta_in: 5.637,  kl_z_ex: 1.353,  kl_z_in: 0.557
epoch: 5\250 (138s),  symKL_DB_eta: 3.786,  symKL_DB_z: 2.807,  loss: -440.788,  ess: 7.774,  kl_eta_ex: 1.072,  kl_eta_in: 1.681,  kl_z_ex: 0.811,  kl_z_in: 0.3

In [5]:
torch.save(enc_z.state_dict(), "../weights/enc-z-%s" % PATH)
torch.save(enc_eta.state_dict(), "../weights/enc-eta-%s" % PATH)
torch.save(oneshot_eta.state_dict(), "../weights/oneshot-eta-%s" % PATH)